In [2]:
!pip install numpy==2.0.0
!pip install protobuf==3.20
!pip install tensorboard
!pip install espnet
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 66.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-datastore 2.21.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
google-cloud-iam 2.19.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
googleapis-common-protos 1.70.0 requires protobuf!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
google-cloud-resource-manager 1.14.2 requires protobuf!=4.21.0,!=4.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.7/524.7 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
from datasets import load_dataset, Audio

low_resource_language_code = "cy" # Welsh
ds = load_dataset("mozilla-foundation/common_voice_11_0", low_resource_language_code)
ds = ds.cast_column("audio", Audio(sampling_rate=16000))

In [2]:
from tqdm import tqdm
import soundfile as sf
import os

train_dataset = []
output_dir = "./speech/train"
os.makedirs(output_dir, exist_ok=True)
for i, sample in tqdm(enumerate(ds["train"]), total=len(ds["train"])):
    waveform = sample["audio"]["array"]
    sr = sample["audio"]["sampling_rate"]
    filename = os.path.join(output_dir, f"{i:05d}.mp3")
    sf.write(filename, waveform, sr, format="mp3")
    train_dataset.append({
        "speech": filename,
        "text": sample["sentence"],
    })

valid_dataset = []
output_dir = "./speech/valid"
os.makedirs(output_dir, exist_ok=True)
for i, sample in tqdm(enumerate(ds["validation"]), total=len(ds["validation"])):
    waveform = sample["audio"]["array"]
    sr = sample["audio"]["sampling_rate"]
    filename = os.path.join(output_dir, f"{i:05d}.mp3")
    sf.write(filename, waveform, sr, format="mp3")
    valid_dataset.append({
        "speech": filename,
        "text": sample["sentence"],
    })

100%|██████████| 5247/5247 [14:10<00:00,  6.17it/s]


## Data Preparation

This notebook follows the data preparation steps outlined in `asr.sh`. Initially, we will create a dump file to store information about the data, including the data ID, audio path, and transcriptions.

ESPnet-EZ supports various types of datasets, including:

1. Dictionary-based dataset with the following structure:
   ```python
   {
     "data_id": {
         "speech": path_to_speech_file,
         "text": transcription
     }
   }
   ```

2. List of datasets with the following structure:
   ```python
   [
     {
         "speech": path_to_speech_file,
         "text": transcription
     }
   ]
   ```

If you choose to use a dictionary-based dataset, it's essential to ensure that each `data_id` is unique. ESPnet-EZ also accepts a dump file that may have already been created by `asr.sh`. However, in this notebook, we will create the dump file from scratch.

Now, let's create dump files!  
Please note that you will need to provide a dictionary to specify the file path and type for each data.
This dictionary should have the following format:

```python
{
    "data_name": ["dump_file_name", "dump_format"]
}
```

In [ ]:
import espnetez as ez

data_info = {
    "speech": ["wav.scp", "sound"],
    "text": ["text", "text"],
}

ez.data.create_dump_file("./dump/train", train_dataset, data_info)
ez.data.create_dump_file("./dump/valid", valid_dataset, data_info)

Now you have dataset files in the `dump` directory.
It looks like this:

wav.scp
```
1255-138279-0008 /hdd/database/librispeech-100/LibriSpeech/dev-other/1255/138279/1255-138279-0008.flac
1255-138279-0022 /hdd/database/librispeech-100/LibriSpeech/dev-other/1255/138279/1255-138279-0022.flac
```

text
```
1255-138279-0008 TWO THREE
1255-138279-0022 IF I SAID SO OF COURSE I WILL
```


## Train sentencepiece model

To train a SentencePiece model, we require a text file for training. Let's begin by creating the training file.

In [ ]:
# generate training texts from the training data
# you can select several datasets to train sentencepiece.
ez.preprocess.prepare_sentences(["dump/train/text"], "dump/spm")

ez.preprocess.train_sentencepiece(
    "dump/spm/train.txt",
    "data/bpemodel",
    vocab_size=5000,
)

## Configure Training Process

For configuring the training process, you can utilize the configuration files already provided by ESPnet contributors. To use a configuration file, you'll need to create a YAML file on your local machine. For instance, you can use the [e-branchformer config](train_asr_e-branchformer_size256_mlp1024_linear1024_e12_mactrue_edrop0.0_ddrop0.0.yaml).

In [ ]:
%%writefile ./train.yml
# network architecture

# encoder related
encoder: transformer
encoder_conf:
    input_layer: "conv2d"
    num_blocks: 2
    linear_units: 64
    dropout_rate: 0.1
    output_size: 8  # dimension of attention
    attention_heads: 2
    attention_dropout_rate: 0.0

# decoder related
decoder: transformer
decoder_conf:
    input_layer: "embed"
    num_blocks: 1
    linear_units: 64
    dropout_rate: 0.1

# hybrid CTC/attention
model_conf:
    ctc_weight: 0.3
    lsm_weight: 0.1
    length_normalized_loss: false

# minibatch related
batch_type: unsorted
batch_size: 8

# optimization related
optim: adam
accum_grad: 2
grad_clip: 5
patience: 0
max_epoch: 10
optim_conf:
    lr: 1.0
scheduler: noamlr
scheduler_conf:
    warmup_steps: 5

# others:
best_model_criterion:
-  - valid
   - acc
   - max
keep_nbest_models: 1

init: xavier_uniform # Our empirical studies shows that this initialization
                     # is very important to low-resource ASR training

Overwriting ./train.yml


In [ ]:
%%writefile ./preprocess.yaml
use_preprocessor: true

token_type: bpe
bpemodel: data/bpemodel/bpe.model
rir_scp: null
rir_apply_prob: 1.0
noise_scp: null
noise_apply_prob: 1.0
noise_db_range: '13_15'
speech_volume_normalize: null
non_linguistic_symbols: null

cleaner: null
g2p: null
preprocessor: default
preprocessor_conf:
  speech_name: speech
  text_name: text

token_list: data/bpemodel/tokens.txt


Overwriting ./preprocess.yaml


## Training

To prepare the stats file before training, you can execute the `collect_stats` method. This step is required before the training process and ensuring accurate statistics for the model.


In [ ]:
import espnetez as ez
import yaml

EXP_DIR = "exp/train_asr_branchformer_e24_amp"
STATS_DIR = "exp/stats"

# load config
# For the configuration, please refer to the last cell in this notebook.
training_config = ez.config.from_yaml(
    "asr",
    "/content/train.yml",
)
with open("/content/preprocess.yaml") as stream:
    preprocessor_config = yaml.safe_load(stream)

training_config.update(preprocessor_config)

with open(preprocessor_config["token_list"], "r") as f:
    training_config["token_list"] = [t.replace("\n", "") for t in f.readlines()]

# When you don't use yaml file, you can load finetune_config in the following way.
# task_class = ez.task.get_ez_task("asr")
# default_config = task_class.get_default_config()
# training_config = default_config.update(your_config_in_dict)

# Define the Trainer class
trainer = ez.Trainer(
    task='asr',
    train_config=training_config,
    train_dump_dir="dump/train",
    valid_dump_dir="dump/valid",
    data_info=data_info,
    output_dir=EXP_DIR,
    stats_dir=STATS_DIR,
    ngpu=0, # number of GPU, change to 0 if run on CPU, 1 if run on GPU (colab)
)
trainer.collect_stats()

/usr/bin/python3 /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-5c9a297a-1254-4d4f-8940-155a854f0aac.json
/usr/local/lib/python3.11/dist-packages/espnet2/schedulers/noam_lr.py:41: UserWarning: NoamLR is deprecated. Use WarmupLR(warmup_steps=5) with Optimizer(lr=0.024999999999999998)
  warnings.warn(


Finally, we are ready to begin the training process!

In [ ]:
trainer.train()

/usr/bin/python3 /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-5c9a297a-1254-4d4f-8940-155a854f0aac.json
/usr/local/lib/python3.11/dist-packages/espnet2/train/trainer.py:609: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(
/usr/local/lib/python3.11/dist-packages/espnet2/asr/espnet_model.py:376: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(False):


## Inference
You can just use the inference API of the ESPnet.

In [ ]:
import librosa
from espnet2.bin.asr_inference import Speech2Text

m = Speech2Text(
    "./exp/train_asr_branchformer_e24_amp/config.yaml",
    "./exp/train_asr_branchformer_e24_amp/valid.acc.best.pth",
    beam_size=10
)

with open("./dump/valid/common-voice-cy/wav.scp", "r") as f:
    sample_path = f.readlines()[0]

with open("./dump/valid/common-voice-cy/text", "r") as f:
    transription = f.readlines()[0]

y, sr = librosa.load(sample_path.split()[1], sr=16000, mono=True)
nbests = m(y)
text, *_ = nbests[0]

In [ ]:
text

'D'

In [ ]:
transription

'0 Cyfrol o ysgrifau cyfoes gan Hafina Clwyd yw Rhywbeth Bob Dydd.\n'